In [3]:
import json
import os, csv
from datasets import Dataset, load_from_disk, concatenate_datasets, load_dataset

import torch
from torch import tensor

import pandas as pd

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('/scratch/gpfs/JHA/mb5157/tokenizers/biomedbert_fast_tokenizer')

from collections import defaultdict, Counter
import numpy as np

/scratch/gpfs/JHA/mb5157/CONDA/envs/graphmert/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
pd.set_option('display.max_colwidth', 600)

# Unite fact-check chunks 

In [84]:
# seq only
prefix = '../'
dataset_path = os.path.join('/projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55')
datasets = []
suffices = [
'accepted_seq_only_qwen3-32b_0-75000_zero_shot',
'accepted_seq_only_qwen3-32b_75000-150000_zero_shot',
'accepted_seq_only_qwen3-32b_150000-225000_zero_shot',
'accepted_seq_only_qwen3-32b_225000-300000_zero_shot',
'accepted_seq_only_qwen3-32b_300000-375000_zero_shot',
'accepted_seq_only_qwen3-32b_375000-450000_zero_shot',
'accepted_seq_only_qwen3-32b_450000-515460_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/accepted_seq_only_qwen3-32b_0-75000_zero_shot
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/accepted_seq_only_qwen3-32b_75000-150000_zero_shot
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/accepted_seq_only_qwen3-32b_150000-225000_zero_shot
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/accepted_seq_only_qwen3-32b_225000-300000_zero_shot
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/accepted_seq_only_qwen3-32b_300000-375000_zero_shot
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/accepted_seq_only_qwen3-32b_375000-450000_zero_shot
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/accepted_seq_only_qwen3-32b_450000-515460_zero_shot
207123


In [85]:
207123 / 515460

0.4018216738447212

In [11]:
# general
prefix = '../'
dataset_path = os.path.join('/projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55')
datasets = []
suffices = [
    'accepted_qwen3-32b_0-75000_zero_shot',
    'accepted_qwen3-32b_75000-150000_zero_shot',
    'accepted_qwen3-32b_150000-225000_zero_shot',
    'accepted_qwen3-32b_225000-300000_zero_shot',
    'accepted_qwen3-32b_300000-375000_zero_shot',
    'accepted_qwen3-32b_375000-450000_zero_shot',
    'accepted_qwen3-32b_450000-515460_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/accepted_qwen3-32b_0-75000_zero_shot
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/accepted_qwen3-32b_75000-150000_zero_shot
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/accepted_qwen3-32b_150000-225000_zero_shot
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/accepted_qwen3-32b_225000-300000_zero_shot
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/accepted_qwen3-32b_300000-375000_zero_shot
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/accepted_qwen3-32b_375000-450000_zero_shot
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/accepted_qwen3-32b_450000-515460_zero_shot
248086


In [14]:
248086 / 515460

0.4812904978077833

### LLM -- all relations

In [58]:
# general
prefix = '../'
dataset_path = os.path.join('/projects/JHA/shared/graph/qwen32b_fixed_rel_kg/')
datasets = []
suffices = [
    'accepted_qwen3-32b_0-75000_zero_shot',
    'accepted_qwen3-32b_75000-150000_zero_shot',
    'accepted_qwen3-32b_150000-225000_zero_shot',
    'accepted_qwen3-32b_225000-289512_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/accepted_qwen3-32b_0-75000_zero_shot
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/accepted_qwen3-32b_75000-150000_zero_shot
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/accepted_qwen3-32b_150000-225000_zero_shot
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/accepted_qwen3-32b_225000-289512_zero_shot
174059


In [ ]:
output_path = os.path.join(dataset_path, f'accepted_{llm_name}_all')
# united_dataset.save_to_disk(output_path)
# print('saved to', output_path)

In [11]:
# based on general knowledge
174059 / 289512

0.6012151482494681

In [7]:
# seq only
prefix = '../'
dataset_path = os.path.join('/projects/JHA/shared/graph/qwen32b_fixed_rel_kg/')
datasets = []
suffices = [
    'accepted_seq_only_qwen3-32b_0-75000_zero_shot',
    'accepted_seq_only_qwen3-32b_75000-150000_zero_shot',
    'accepted_seq_only_qwen3-32b_150000-225000_zero_shot',
    'accepted_seq_only_qwen3-32b_225000-289512_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/accepted_seq_only_qwen3-32b_0-75000_zero_shot
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/accepted_seq_only_qwen3-32b_75000-150000_zero_shot
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/accepted_seq_only_qwen3-32b_150000-225000_zero_shot
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/accepted_seq_only_qwen3-32b_225000-289512_zero_shot
171350


In [5]:
# based on sequence only
171350 / 289512

0.5918580231562077

#### diff

In [12]:
# seq only

prefix = '../'
dataset_path = os.path.join('/projects/JHA/shared/graph/qwen32b_fixed_rel_kg/')
datasets = []
suffices = [
    'accepted_seq_only_qwen3-32b_0-75000_zero_shot_diff',
    'accepted_seq_only_qwen3-32b_75000-150000_zero_shot_diff',
    'accepted_seq_only_qwen3-32b_150000-225000_zero_shot_diff',
    'accepted_seq_only_qwen3-32b_225000-300000_zero_shot_diff',
    'accepted_seq_only_qwen3-32b_300000-324519_zero_shot_diff',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/accepted_seq_only_qwen3-32b_0-75000_zero_shot_diff
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/accepted_seq_only_qwen3-32b_75000-150000_zero_shot_diff
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/accepted_seq_only_qwen3-32b_150000-225000_zero_shot_diff
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/accepted_seq_only_qwen3-32b_225000-300000_zero_shot_diff
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/accepted_seq_only_qwen3-32b_300000-324519_zero_shot_diff
66656


In [13]:
# based on sequence only
(171350 + 66656) / (289512 + 324519)

0.3876123518193707

In [103]:
324519 / 75_000

4.32692

In [59]:
# general knowledge 
prefix = '../'
dataset_path = os.path.join('/projects/JHA/shared/graph/qwen32b_fixed_rel_kg/')
datasets = []
suffices = [
    'accepted_qwen3-32b_0-75000_zero_shot_diff',
    'accepted_qwen3-32b_75000-150000_zero_shot_diff',
    'accepted_qwen3-32b_150000-225000_zero_shot_diff',
    'accepted_qwen3-32b_225000-300000_zero_shot_diff',
    'accepted_qwen3-32b_300000-324519_zero_shot_diff',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/accepted_qwen3-32b_0-75000_zero_shot_diff
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/accepted_qwen3-32b_75000-150000_zero_shot_diff
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/accepted_qwen3-32b_150000-225000_zero_shot_diff
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/accepted_qwen3-32b_225000-300000_zero_shot_diff
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/accepted_qwen3-32b_300000-324519_zero_shot_diff
108790


### GraphMERT

In [5]:
keys_duplicates = ['id', 'head', 'relation', 'tail']

accepted_* -- based on sequence + general knowledge;

accepted_seq_only -- based on sequence;

In [4]:
dataset_path = '../outputs/350abstracts/qwen32/score0.55/span/rel29/bs256_lr_0.0004/predictions/top_20'
datasets = []
suffices = [
    'accepted_qwen3-32b_0-100000_zero_shot',
    'accepted_qwen3-32b_100000-139565_zero_shot'
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from ../outputs/350abstracts/qwen32/score0.55/span/rel29/bs256_lr_0.0004/predictions/top_20/accepted_qwen3-32b_0-100000_zero_shot
loaded from ../outputs/350abstracts/qwen32/score0.55/span/rel29/bs256_lr_0.0004/predictions/top_20/accepted_qwen3-32b_100000-139565_zero_shot
100765


In [6]:
united_dataset = united_dataset.to_pandas()
united_dataset_deduplicated = united_dataset.drop_duplicates(subset=keys_duplicates)

whole_dataset = pd.read_csv('/projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/qwen32b_beta_0.67.csv')
whole_dataset_deduplicated = whole_dataset.drop_duplicates(subset=keys_duplicates)

fact_score = len(united_dataset_deduplicated) / len(whole_dataset_deduplicated)
print(round(fact_score, 3))

0.722


In [17]:
united_dataset_deduplicated.iloc[8:20]

,Unnamed: 0,id,text,head,relation,tail
8,10,346498,"90 % ci were 132. 1 % ( 122. 7 – 142. 3 ) for auc 0 – t, and 167. 6 % ( 154. 6 – 181. 8 ) for c max. on the other hand, sildenafil pharmacokinetics were not affected. g max changed from 93. 7 mg / dl to 95. 6 mg / dl ( p > 0. 001 ) and auc g0 - t from 512. 8 ng. h / ml to 532. 75 ng. h / ml ( p > 0. 001 ) after co - administration of both drugs. conclusion sildenafil significantly affected the pharmacokinetic parameters of saxagliptin when co",sildenafil,cause_of,pharmacokinetics of saxagliptin
9,11,704648,". 001 ). increased pp was found in 24. 2 % of the entire study population and was higher in patients with osa compared to no ‐ osa group ( 27. 5 % vs 22. 4 %, p <. 0001 ). in an unadjusted logistic regression model, osa was associated with a 1. 3 ‐ fold higher risk of having increased pp ( 95 % ci 1. 16 ‐ 1. 48, p <. 001 ). in a multivariable ‐ adjusted model, higher age, male sex, and history of hypertension, but not osa ( or 0. 89 ; 95 % ci 0. 77 ‐ 1. 02",increased pp,associated_with,osa
10,12,606314,"in obesity, the content of bat and beige adipose tissue declines as these tissues acquire the wat characteristics via the process called “ whitening ”. this process has been rarely explored for its implication in obesity, whether it contributes to or exacerbates obesity. emerging research has demonstrated that bat / beige adipose tissue whitening is a sophisticated metabolic complication of obesity that is linked to multiple factors. the current review provides clarification on the influence of various factors such as diet, age, genetics, thermoneutrality, and chemical exposure on bat / beige adipose tissue whitening. moreover, the defects and mechanisms that underpin the whitening",bat / beige adipose tissue whitening,associated_with,obesity
11,13,404552,"in adipogenesis. methods this study used the 3t3 ‐ l1 cell line and human visceral adipose tissue ( vat ) ‐ derived adipocytes to determine the role of mir ‐ 182 in adipogenesis. adipose tissues from mice with high ‐ fat diet – induced obesity, ob / ob mice, or human individuals with obesity were used to determine the association of mir ‐ 182 levels with obesity. a luciferase reporter assay was used to determine the target of mir ‐ 182. results the expression level of mir ‐ 182 was greatly downregulated during white adipogenesis and markedly lower in the vat of mice and humans with obesity. ectopic expression of mir ‐ 182 in 3t3 ‐ l1 cells and",mir ‐ 182,associated_with,adipocyte
12,15,270012,"many human diseases such as cancer, neurodegeneration, obesity, diabetes, infection, and aging. although its acute post - translational regulation is well described, recent studies have also shown that autophagy can be controlled at the transcriptional and post - transcriptional levels. nuclear receptors ( nrs ) are in general ligand - dependent transcription factors consisting of 48 members in humans. these receptors extensively control transcription of a variety of genes involved in development, metabolism, and inflammation. in this review, we discuss the roles and mechanisms of nrs in an aspect of transcriptional regulation of hepatic autophagy, and how the nr - driven autophagy pathway can be harnessed to treat various liver diseases",nuclear receptors,plays_role,hepatic autophagy
13,16,33486,"in diabetes distress ( all p > 0. 05 ). when change in diabetes acceptance from baseline to follow - up was added to the model, improved diabetes distress was explained by increasing diabetes acceptance ( β = 0. 41, p < 0. 01 ) and a shorter duration of diabetes ( β = - 0. 18, p = 0. 03 ), while baseline diabetes acceptance was no longer significantly associated ( β = −0. 14, p > 0. 05 ). conclusions diabetes acceptance is inversely related to diabetes distress, and increasing acceptance explained greater improvement in diabetes distress. these findings suggest that increasing diabetes acceptance may facilitate the reduction",diabetes dist

In [16]:
whole_dataset_deduplicated.iloc[10:]

,Unnamed: 0,id,text,head,relation,tail
10,10,346498,"90 % ci were 132. 1 % ( 122. 7 – 142. 3 ) for auc 0 – t, and 167. 6 % ( 154. 6 – 181. 8 ) for c max. on the other hand, sildenafil pharmacokinetics were not affected. g max changed from 93. 7 mg / dl to 95. 6 mg / dl ( p > 0. 001 ) and auc g0 - t from 512. 8 ng. h / ml to 532. 75 ng. h / ml ( p > 0. 001 ) after co - administration of both drugs. conclusion sildenafil significantly affected the pharmacokinetic parameters of saxagliptin when co",sildenafil,cause_of,pharmacokinetics of saxagliptin
11,11,704648,". 001 ). increased pp was found in 24. 2 % of the entire study population and was higher in patients with osa compared to no ‐ osa group ( 27. 5 % vs 22. 4 %, p <. 0001 ). in an unadjusted logistic regression model, osa was associated with a 1. 3 ‐ fold higher risk of having increased pp ( 95 % ci 1. 16 ‐ 1. 48, p <. 001 ). in a multivariable ‐ adjusted model, higher age, male sex, and history of hypertension, but not osa ( or 0. 89 ; 95 % ci 0. 77 ‐ 1. 02",increased pp,associated_with,osa
12,12,606314,"in obesity, the content of bat and beige adipose tissue declines as these tissues acquire the wat characteristics via the process called “ whitening ”. this process has been rarely explored for its implication in obesity, whether it contributes to or exacerbates obesity. emerging research has demonstrated that bat / beige adipose tissue whitening is a sophisticated metabolic complication of obesity that is linked to multiple factors. the current review provides clarification on the influence of various factors such as diet, age, genetics, thermoneutrality, and chemical exposure on bat / beige adipose tissue whitening. moreover, the defects and mechanisms that underpin the whitening",bat / beige adipose tissue whitening,associated_with,obesity
13,13,404552,"in adipogenesis. methods this study used the 3t3 ‐ l1 cell line and human visceral adipose tissue ( vat ) ‐ derived adipocytes to determine the role of mir ‐ 182 in adipogenesis. adipose tissues from mice with high ‐ fat diet – induced obesity, ob / ob mice, or human individuals with obesity were used to determine the association of mir ‐ 182 levels with obesity. a luciferase reporter assay was used to determine the target of mir ‐ 182. results the expression level of mir ‐ 182 was greatly downregulated during white adipogenesis and markedly lower in the vat of mice and humans with obesity. ectopic expression of mir ‐ 182 in 3t3 ‐ l1 cells and",mir ‐ 182,associated_with,adipocyte
14,14,124712,"and are warranted. many studies have reported that metabolic dysfunction is closely involved in the complex mechanism underlying the development of non - alcoholic fatty liver disease ( nafld ), which has prompted a movement to consider renaming nafld as metabolic dysfunction - associated fatty liver disease ( mafld ). metabolic dysfunction in this context encompasses obesity, type 2 diabetes mellitus, hypertension, dyslipidemia, and metabolic syndrome, with insulin resistance as the common underlying pathophysiology. imbalance between energy intake and expenditure results in insulin resistance in various tissues and alteration of the gut microbiota, resulting in fat accumulation in the liver. the role of genetics has also been revealed in hepatic fat accumulation and",metabolic dysfunction - associated fatty liver disease ( mafld ),due_to,metabolic dysfunction
...,...,...,...,...,...,...
139560,139560,374992,"function, is a critical role played by the brain ’ s hypothalamic region. specifically, the paraventricular nucleus of the hypothalamus ( pvn ) regulates autonomic functions related to metabolism, such as hepatic glucose production. although insulin can act directly on hepatic tissue to inhibit hepatic glucose production, recent evidence implicates that central actions of insulin within pvn also regulate glucose metabolism. however, specific central circuits responsible for insulin signaling with relation to hepatic regulati

In [39]:
dataset_path = '../outputs/350abstracts/qwen32/score0.55/span/rel29/bs256_lr_0.0004/predictions/top_20'
datasets = []
suffices = [
    'accepted_seq_only_qwen3-32b_0-100000_zero_shot',
    'accepted_seq_only_qwen3-32b_100000-139565_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from ../outputs/350abstracts/qwen32/score0.55/span/rel29/bs256_lr_0.0004/predictions/top_20/accepted_seq_only_qwen3-32b_0-100000_zero_shot
loaded from ../outputs/350abstracts/qwen32/score0.55/span/rel29/bs256_lr_0.0004/predictions/top_20/accepted_seq_only_qwen3-32b_100000-139565_zero_shot
97379


In [40]:
united_dataset = united_dataset.to_pandas()
united_dataset_deduplicated = united_dataset.drop_duplicates(subset=keys_duplicates)

whole_dataset = pd.read_csv('/projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/qwen32b_beta_0.67.csv')
whole_dataset_deduplicated = whole_dataset.drop_duplicates(subset=keys_duplicates)

fact_score = len(united_dataset_deduplicated) / len(whole_dataset_deduplicated)
print(round(fact_score, 3))

0.698


In [41]:
len(whole_dataset_deduplicated), len(whole_dataset)

(138201, 139565)

#### deduplicated -- del these chunks

In [48]:
# based on seq only knowledge
dataset_path = '../outputs/350abstracts/qwen32/score0.55/span/rel29/bs256_lr_0.0004/predictions/top_20'
datasets = []
suffices = [
'accepted_seq_only_qwen3-32b_0-100000_zero_shot_deduplicated',
'accepted_seq_only_qwen3-32b_100000-109293_zero_shot_deduplicated',

]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from ../outputs/350abstracts/qwen32/score0.55/span/rel29/bs256_lr_0.0004/predictions/top_20/accepted_seq_only_qwen3-32b_0-100000_zero_shot_deduplicated
loaded from ../outputs/350abstracts/qwen32/score0.55/span/rel29/bs256_lr_0.0004/predictions/top_20/accepted_seq_only_qwen3-32b_100000-109293_zero_shot_deduplicated
74582


In [50]:
# based on general knowledge
dataset_path = '../outputs/350abstracts/qwen32/score0.55/span/rel29/bs256_lr_0.0004/predictions/top_20'
datasets = []
suffices = [
'accepted_qwen3-32b_0-100000_zero_shot_deduplicated',
'accepted_qwen3-32b_100000-109293_zero_shot_deduplicated',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from ../outputs/350abstracts/qwen32/score0.55/span/rel29/bs256_lr_0.0004/predictions/top_20/accepted_qwen3-32b_0-100000_zero_shot_deduplicated
loaded from ../outputs/350abstracts/qwen32/score0.55/span/rel29/bs256_lr_0.0004/predictions/top_20/accepted_qwen3-32b_100000-109293_zero_shot_deduplicated
76108


### GraphMERT ablations

In [42]:


# based on general knowledge
dataset_path = '/projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_hgat/cleaned_graphs'
datasets = []
suffices = [
    'accepted_qwen3-32b_0-50000_zero_shot',
    'accepted_qwen3-32b_50000-100000_zero_shot',
    'accepted_qwen3-32b_100000-150000_zero_shot',
    'accepted_qwen3-32b_150000-167443_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_hgat/cleaned_graphs/accepted_qwen3-32b_0-50000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_hgat/cleaned_graphs/accepted_qwen3-32b_50000-100000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_hgat/cleaned_graphs/accepted_qwen3-32b_100000-150000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_hgat/cleaned_graphs/accepted_qwen3-32b_150000-167443_zero_shot
117194


In [43]:
united_dataset = united_dataset.to_pandas()
united_dataset_deduplicated = united_dataset.drop_duplicates(subset=keys_duplicates)

whole_dataset = pd.read_csv('/projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_hgat/cleaned_graphs/qwen32b_beta_0.67.csv')
whole_dataset_deduplicated = whole_dataset.drop_duplicates(subset=keys_duplicates)

fact_score = len(united_dataset_deduplicated) / len(whole_dataset_deduplicated)
print(round(fact_score, 3))

0.699


In [48]:
len(whole_dataset_deduplicated), len(whole_dataset)

(155311, 167443)

In [49]:
# based on sequence
dataset_path = '/projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_hgat/cleaned_graphs'
datasets = []
suffices = [
    'accepted_seq_only_qwen3-32b_0-50000_zero_shot',
    'accepted_seq_only_qwen3-32b_50000-100000_zero_shot',
    'accepted_seq_only_qwen3-32b_100000-150000_zero_shot',
    'accepted_seq_only_qwen3-32b_150000-167443_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_hgat/cleaned_graphs/accepted_seq_only_qwen3-32b_0-50000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_hgat/cleaned_graphs/accepted_seq_only_qwen3-32b_50000-100000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_hgat/cleaned_graphs/accepted_seq_only_qwen3-32b_100000-150000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_hgat/cleaned_graphs/accepted_seq_only_qwen3-32b_150000-167443_zero_shot
114349


In [50]:
united_dataset = united_dataset.to_pandas()
united_dataset_deduplicated = united_dataset.drop_duplicates(subset=keys_duplicates)

whole_dataset = pd.read_csv(os.path.join(dataset_path, 'qwen32b_beta_0.67.csv'))
whole_dataset_deduplicated = whole_dataset.drop_duplicates(subset=keys_duplicates)

fact_score = len(united_dataset_deduplicated) / len(whole_dataset_deduplicated)
print(round(fact_score, 3))

0.683


In [51]:
len(whole_dataset_deduplicated), len(whole_dataset)

(155311, 167443)

In [52]:
# based on sequence
dataset_path = '/projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_dropout/cleaned_graphs'
datasets = []
suffices = [
    'accepted_seq_only_qwen3-32b_0-50000_zero_shot',
    'accepted_seq_only_qwen3-32b_50000-100000_zero_shot',
    'accepted_seq_only_qwen3-32b_100000-149952_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_dropout/cleaned_graphs/accepted_seq_only_qwen3-32b_0-50000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_dropout/cleaned_graphs/accepted_seq_only_qwen3-32b_50000-100000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_dropout/cleaned_graphs/accepted_seq_only_qwen3-32b_100000-149952_zero_shot
103372


In [53]:
united_dataset = united_dataset.to_pandas()
united_dataset_deduplicated = united_dataset.drop_duplicates(subset=keys_duplicates)

whole_dataset = pd.read_csv(os.path.join(dataset_path, 'qwen32b_beta_0.67.csv'))
whole_dataset_deduplicated = whole_dataset.drop_duplicates(subset=keys_duplicates)

fact_score = len(united_dataset_deduplicated) / len(whole_dataset_deduplicated)
print(round(fact_score, 3))

0.69


In [54]:
len(whole_dataset_deduplicated), len(whole_dataset)

(138363, 149952)

In [55]:
103372 / 149952

0.6893672641912079

In [56]:
# based on general knowledge
dataset_path = '/projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_dropout/cleaned_graphs'
datasets = []
suffices = [
    'accepted_qwen3-32b_0-50000_zero_shot',
    'accepted_qwen3-32b_50000-100000_zero_shot',
    'accepted_qwen3-32b_100000-149952_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_dropout/cleaned_graphs/accepted_qwen3-32b_0-50000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_dropout/cleaned_graphs/accepted_qwen3-32b_50000-100000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_dropout/cleaned_graphs/accepted_qwen3-32b_100000-149952_zero_shot
106281


In [57]:
united_dataset = united_dataset.to_pandas()
united_dataset_deduplicated = united_dataset.drop_duplicates(subset=keys_duplicates)

whole_dataset = pd.read_csv(os.path.join(dataset_path, 'qwen32b_beta_0.67.csv'))
whole_dataset_deduplicated = whole_dataset.drop_duplicates(subset=keys_duplicates)

fact_score = len(united_dataset_deduplicated) / len(whole_dataset_deduplicated)
print(round(fact_score, 3))

0.709


In [58]:
len(whole_dataset_deduplicated), len(whole_dataset)

(138363, 149952)

In [65]:
# based on general knowledge
dataset_path = '/projects/JHA/shared/dataset/qwen32b/alpha_0.55/mlm'
datasets = []
suffices = [
    'accepted_qwen3-32b_0-50000_zero_shot',
    'accepted_qwen3-32b_50000-100000_zero_shot',
    'accepted_qwen3-32b_100000-150000_zero_shot',
    'accepted_qwen3-32b_150000-188211_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/mlm/accepted_qwen3-32b_0-50000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/mlm/accepted_qwen3-32b_50000-100000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/mlm/accepted_qwen3-32b_100000-150000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/mlm/accepted_qwen3-32b_150000-188211_zero_shot
135906


In [66]:
united_dataset = united_dataset.to_pandas()
united_dataset_deduplicated = united_dataset.drop_duplicates(subset=keys_duplicates)

whole_dataset = pd.read_csv(os.path.join(dataset_path, 'cleaned_graphs/qwen32b_beta_0.67.csv'))
whole_dataset_deduplicated = whole_dataset.drop_duplicates(subset=keys_duplicates)

fact_score = len(united_dataset_deduplicated) / len(whole_dataset_deduplicated)
print(round(fact_score, 3))

0.722


In [67]:
len(whole_dataset_deduplicated), len(whole_dataset)

(185768, 188211)

In [68]:
# based on sequence
dataset_path = '/projects/JHA/shared/dataset/qwen32b/alpha_0.55/mlm'
datasets = []
suffices = [
    'accepted_seq_only_qwen3-32b_0-50000_zero_shot',
    'accepted_seq_only_qwen3-32b_50000-100000_zero_shot',
    'accepted_seq_only_qwen3-32b_100000-150000_zero_shot',
    'accepted_seq_only_qwen3-32b_150000-188211_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/mlm/accepted_seq_only_qwen3-32b_0-50000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/mlm/accepted_seq_only_qwen3-32b_50000-100000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/mlm/accepted_seq_only_qwen3-32b_100000-150000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/mlm/accepted_seq_only_qwen3-32b_150000-188211_zero_shot
129995


In [69]:
united_dataset = united_dataset.to_pandas()
united_dataset_deduplicated = united_dataset.drop_duplicates(subset=keys_duplicates)

whole_dataset = pd.read_csv(os.path.join(dataset_path, 'cleaned_graphs/qwen32b_beta_0.67.csv'))
whole_dataset_deduplicated = whole_dataset.drop_duplicates(subset=keys_duplicates)

fact_score = len(united_dataset_deduplicated) / len(whole_dataset_deduplicated)
print(round(fact_score, 3))

0.691


In [70]:
len(whole_dataset_deduplicated), len(whole_dataset)

(185768, 188211)

### Expanded & LLM KGs as seed KG abaltion

#### before cleaning

In [88]:
dataset_path = '/projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/expanded_kg/alpha_0.55/cleaned_graphs'
datasets = []
suffices = [
'accepted_qwen3-32b_0-50000_zero_shot',
'accepted_qwen3-32b_50000-100000_zero_shot',
'accepted_qwen3-32b_100000-150000_zero_shot',
'accepted_qwen3-32b_150000-200000_zero_shot',
'accepted_qwen3-32b_200000-250000_zero_shot',
'accepted_qwen3-32b_250000-300000_zero_shot',
'accepted_qwen3-32b_300000-350000_zero_shot',
'accepted_qwen3-32b_350000-400000_zero_shot',
'accepted_qwen3-32b_400000-450000_zero_shot',
'accepted_qwen3-32b_450000-500000_zero_shot',
'accepted_qwen3-32b_500000-530875_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/expanded_kg/alpha_0.55/cleaned_graphs/accepted_qwen3-32b_0-50000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/expanded_kg/alpha_0.55/cleaned_graphs/accepted_qwen3-32b_50000-100000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/expanded_kg/alpha_0.55/cleaned_graphs/accepted_qwen3-32b_100000-150000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/expanded_kg/alpha_0.55/cleaned_graphs/accepted_qwen3-32b_150000-200000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/expanded_kg/alpha_0.55/cleaned_graphs/accepted_qwen3-32b_200000-250000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/expanded_kg/alpha_0.55/cleaned_graphs/accepted_qwen3-32b_250000-300000_zero_shot
loaded from /projects/J

In [89]:
united_dataset = united_dataset.to_pandas()
united_dataset_deduplicated = united_dataset.drop_duplicates(subset=keys_duplicates)

whole_dataset = pd.read_csv(os.path.join(dataset_path, 'qwen32b_beta_0.67.csv'))
whole_dataset_deduplicated = whole_dataset.drop_duplicates(subset=keys_duplicates)

fact_score = len(united_dataset_deduplicated) / len(whole_dataset_deduplicated)
print(round(fact_score, 3))

0.79


In [90]:
len(whole_dataset_deduplicated), len(whole_dataset)

(492271, 530875)

In [133]:
dataset_path = '/projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/expanded_kg/alpha_0.55/cleaned_graphs'
datasets = []
suffices = [
'accepted_seq_only_qwen3-32b_0-50000_zero_shot',
'accepted_seq_only_qwen3-32b_50000-100000_zero_shot',
'accepted_seq_only_qwen3-32b_100000-150000_zero_shot',
'accepted_seq_only_qwen3-32b_150000-200000_zero_shot',
'accepted_seq_only_qwen3-32b_200000-250000_zero_shot',
'accepted_seq_only_qwen3-32b_250000-300000_zero_shot',
'accepted_seq_only_qwen3-32b_300000-350000_zero_shot',
'accepted_seq_only_qwen3-32b_350000-400000_zero_shot',
'accepted_seq_only_qwen3-32b_400000-450000_zero_shot',
'accepted_seq_only_qwen3-32b_450000-500000_zero_shot',
'accepted_seq_only_qwen3-32b_500000-530875_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/expanded_kg/alpha_0.55/cleaned_graphs/accepted_seq_only_qwen3-32b_0-50000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/expanded_kg/alpha_0.55/cleaned_graphs/accepted_seq_only_qwen3-32b_50000-100000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/expanded_kg/alpha_0.55/cleaned_graphs/accepted_seq_only_qwen3-32b_100000-150000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/expanded_kg/alpha_0.55/cleaned_graphs/accepted_seq_only_qwen3-32b_150000-200000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/expanded_kg/alpha_0.55/cleaned_graphs/accepted_seq_only_qwen3-32b_200000-250000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/expanded_kg/alpha_0.55/cleaned_graphs/accepted_seq_only_qwe

In [134]:
united_dataset = united_dataset.to_pandas()
united_dataset_deduplicated = united_dataset.drop_duplicates(subset=keys_duplicates)

whole_dataset = pd.read_csv(os.path.join(dataset_path, 'qwen32b_beta_0.67.csv'))
whole_dataset_deduplicated = whole_dataset.drop_duplicates(subset=keys_duplicates)

fact_score = len(united_dataset_deduplicated) / len(whole_dataset_deduplicated)
print(round(fact_score, 3))

0.77


In [135]:
len(whole_dataset_deduplicated), len(whole_dataset)

(492271, 530875)

In [116]:
dataset_path = '/projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/llm_kg/alpha_0.55/cleaned_graphs'
datasets = []
suffices = [
'accepted_qwen3-32b_0-50000_zero_shot',
'accepted_qwen3-32b_50000-100000_zero_shot',
'accepted_qwen3-32b_100000-150000_zero_shot',
'accepted_qwen3-32b_150000-200000_zero_shot',
'accepted_qwen3-32b_200000-250000_zero_shot',
'accepted_qwen3-32b_250000-300000_zero_shot',
'accepted_qwen3-32b_300000-350000_zero_shot',
'accepted_qwen3-32b_350000-400000_zero_shot',
'accepted_qwen3-32b_400000-450000_zero_shot',
'accepted_qwen3-32b_450000-500000_zero_shot',
'accepted_qwen3-32b_500000-550000_zero_shot',
'accepted_qwen3-32b_550000-600000_zero_shot',
'accepted_qwen3-32b_600000-646350_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/llm_kg/alpha_0.55/cleaned_graphs/accepted_qwen3-32b_0-50000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/llm_kg/alpha_0.55/cleaned_graphs/accepted_qwen3-32b_50000-100000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/llm_kg/alpha_0.55/cleaned_graphs/accepted_qwen3-32b_100000-150000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/llm_kg/alpha_0.55/cleaned_graphs/accepted_qwen3-32b_150000-200000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/llm_kg/alpha_0.55/cleaned_graphs/accepted_qwen3-32b_200000-250000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/llm_kg/alpha_0.55/cleaned_graphs/accepted_qwen3-32b_250000-300000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed

In [117]:
united_dataset = united_dataset.to_pandas()
united_dataset_deduplicated = united_dataset.drop_duplicates(subset=keys_duplicates)

whole_dataset = pd.read_csv(os.path.join(dataset_path, 'qwen32b_beta_0.67.csv'))
whole_dataset_deduplicated = whole_dataset.drop_duplicates(subset=keys_duplicates)

fact_score = len(united_dataset_deduplicated) / len(whole_dataset_deduplicated)
print(round(fact_score, 3))

0.767


In [118]:
len(whole_dataset_deduplicated), len(whole_dataset)

(589338, 646350)

In [93]:
dataset_path = '/projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/llm_kg/alpha_0.55/cleaned_graphs'
datasets = []
suffices = [
'accepted_seq_only_qwen3-32b_0-50000_zero_shot',
'accepted_seq_only_qwen3-32b_50000-100000_zero_shot',
'accepted_seq_only_qwen3-32b_100000-150000_zero_shot',
'accepted_seq_only_qwen3-32b_150000-200000_zero_shot',
'accepted_seq_only_qwen3-32b_200000-250000_zero_shot',
'accepted_seq_only_qwen3-32b_250000-300000_zero_shot',
'accepted_seq_only_qwen3-32b_300000-350000_zero_shot',
'accepted_seq_only_qwen3-32b_350000-400000_zero_shot',
'accepted_seq_only_qwen3-32b_400000-450000_zero_shot',
'accepted_seq_only_qwen3-32b_450000-500000_zero_shot',
'accepted_seq_only_qwen3-32b_500000-550000_zero_shot',
'accepted_seq_only_qwen3-32b_550000-600000_zero_shot',
'accepted_seq_only_qwen3-32b_600000-646350_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/llm_kg/alpha_0.55/cleaned_graphs/accepted_seq_only_qwen3-32b_0-50000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/llm_kg/alpha_0.55/cleaned_graphs/accepted_seq_only_qwen3-32b_50000-100000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/llm_kg/alpha_0.55/cleaned_graphs/accepted_seq_only_qwen3-32b_100000-150000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/llm_kg/alpha_0.55/cleaned_graphs/accepted_seq_only_qwen3-32b_150000-200000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/llm_kg/alpha_0.55/cleaned_graphs/accepted_seq_only_qwen3-32b_200000-250000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/llm_kg/alpha_0.55/cleaned_graphs/accepted_seq_only_qwen3-32b_250000-300000_zero_shot

In [94]:
united_dataset = united_dataset.to_pandas()
united_dataset_deduplicated = united_dataset.drop_duplicates(subset=keys_duplicates)

whole_dataset = pd.read_csv(os.path.join(dataset_path, 'qwen32b_beta_0.67.csv'))
whole_dataset_deduplicated = whole_dataset.drop_duplicates(subset=keys_duplicates)

fact_score = len(united_dataset_deduplicated) / len(whole_dataset_deduplicated)
print(round(fact_score, 3))

0.744


In [95]:
len(whole_dataset_deduplicated), len(whole_dataset)

(589338, 646350)

#### after cleaning

In [136]:
dataset_path = '/projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/expanded_kg/alpha_0.55/cleaned_graphs'
datasets = []
suffices = [
'accepted_seq_only_qwen3-32b_0-50000_zero_shot',
'accepted_seq_only_qwen3-32b_50000-100000_zero_shot',
'accepted_seq_only_qwen3-32b_100000-150000_zero_shot',
'accepted_seq_only_qwen3-32b_150000-200000_zero_shot',
'accepted_seq_only_qwen3-32b_200000-250000_zero_shot',
'accepted_seq_only_qwen3-32b_250000-300000_zero_shot',
'accepted_seq_only_qwen3-32b_300000-350000_zero_shot',
'accepted_seq_only_qwen3-32b_350000-400000_zero_shot',
'accepted_seq_only_qwen3-32b_400000-450000_zero_shot',
'accepted_seq_only_qwen3-32b_450000-500000_zero_shot',
'accepted_seq_only_qwen3-32b_500000-550000_zero_shot',
'accepted_seq_only_qwen3-32b_550000-600000_zero_shot',
'accepted_seq_only_qwen3-32b_600000-650000_zero_shot',
'accepted_seq_only_qwen3-32b_650000-690526_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/expanded_kg/alpha_0.55/cleaned_graphs/accepted_seq_only_qwen3-32b_0-50000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/expanded_kg/alpha_0.55/cleaned_graphs/accepted_seq_only_qwen3-32b_50000-100000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/expanded_kg/alpha_0.55/cleaned_graphs/accepted_seq_only_qwen3-32b_100000-150000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/expanded_kg/alpha_0.55/cleaned_graphs/accepted_seq_only_qwen3-32b_150000-200000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/expanded_kg/alpha_0.55/cleaned_graphs/accepted_seq_only_qwen3-32b_200000-250000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/expanded_kg/alpha_0.55/cleaned_graphs/accepted_seq_only_qwen3-32b_250000-300000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablation

In [137]:
united_dataset = united_dataset.to_pandas()
united_dataset_deduplicated = united_dataset.drop_duplicates(subset=keys_duplicates)

whole_dataset = pd.read_csv(os.path.join(dataset_path, 'qwen32b_beta_0.67.csv'))
whole_dataset_deduplicated = whole_dataset.drop_duplicates(subset=keys_duplicates)

fact_score = len(united_dataset_deduplicated) / len(whole_dataset_deduplicated)
print(round(fact_score, 3))

0.77


In [138]:
len(whole_dataset_deduplicated), len(whole_dataset)

(614503, 690526)

In [122]:
dataset_path = '/projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/expanded_kg/alpha_0.55/cleaned_graphs'
datasets = []
suffices = [
'accepted_qwen3-32b_0-50000_zero_shot',
'accepted_qwen3-32b_50000-100000_zero_shot',
'accepted_qwen3-32b_100000-150000_zero_shot',
'accepted_qwen3-32b_150000-200000_zero_shot',
'accepted_qwen3-32b_200000-250000_zero_shot',
'accepted_qwen3-32b_250000-300000_zero_shot',
'accepted_qwen3-32b_300000-350000_zero_shot',
'accepted_qwen3-32b_350000-400000_zero_shot',
'accepted_qwen3-32b_400000-450000_zero_shot',
'accepted_qwen3-32b_450000-500000_zero_shot',
'accepted_qwen3-32b_500000-550000_zero_shot',
'accepted_qwen3-32b_550000-600000_zero_shot',
'accepted_qwen3-32b_600000-650000_zero_shot',
'accepted_qwen3-32b_650000-690526_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/expanded_kg/alpha_0.55/cleaned_graphs/accepted_qwen3-32b_0-50000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/expanded_kg/alpha_0.55/cleaned_graphs/accepted_qwen3-32b_50000-100000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/expanded_kg/alpha_0.55/cleaned_graphs/accepted_qwen3-32b_100000-150000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/expanded_kg/alpha_0.55/cleaned_graphs/accepted_qwen3-32b_150000-200000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/expanded_kg/alpha_0.55/cleaned_graphs/accepted_qwen3-32b_200000-250000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/expanded_kg/alpha_0.55/cleaned_graphs/accepted_qwen3-32b_250000-300000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/expanded_kg/alpha_0.55/cleaned_graphs/accepted_qwen3

In [123]:
united_dataset = united_dataset.to_pandas()
united_dataset_deduplicated = united_dataset.drop_duplicates(subset=keys_duplicates)

whole_dataset = pd.read_csv(os.path.join(dataset_path, 'qwen32b_beta_0.67.csv'))
whole_dataset_deduplicated = whole_dataset.drop_duplicates(subset=keys_duplicates)

fact_score = len(united_dataset_deduplicated) / len(whole_dataset_deduplicated)
print(round(fact_score, 3))

0.791


In [124]:
len(whole_dataset_deduplicated), len(whole_dataset)

(614503, 690526)

In [139]:
dataset_path = '/projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/llm_kg/alpha_0.55/cleaned_graphs'
datasets = []
suffices = [
'accepted_seq_only_qwen3-32b_0-50000_zero_shot',
'accepted_seq_only_qwen3-32b_50000-100000_zero_shot',
'accepted_seq_only_qwen3-32b_100000-150000_zero_shot',
'accepted_seq_only_qwen3-32b_150000-200000_zero_shot',
'accepted_seq_only_qwen3-32b_200000-250000_zero_shot',
'accepted_seq_only_qwen3-32b_250000-300000_zero_shot',
'accepted_seq_only_qwen3-32b_300000-350000_zero_shot',
'accepted_seq_only_qwen3-32b_350000-400000_zero_shot',
'accepted_seq_only_qwen3-32b_400000-450000_zero_shot',
'accepted_seq_only_qwen3-32b_450000-500000_zero_shot',
'accepted_seq_only_qwen3-32b_500000-550000_zero_shot',
'accepted_seq_only_qwen3-32b_550000-580703_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/llm_kg/alpha_0.55/cleaned_graphs/accepted_seq_only_qwen3-32b_0-50000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/llm_kg/alpha_0.55/cleaned_graphs/accepted_seq_only_qwen3-32b_50000-100000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/llm_kg/alpha_0.55/cleaned_graphs/accepted_seq_only_qwen3-32b_100000-150000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/llm_kg/alpha_0.55/cleaned_graphs/accepted_seq_only_qwen3-32b_150000-200000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/llm_kg/alpha_0.55/cleaned_graphs/accepted_seq_only_qwen3-32b_200000-250000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/llm_kg/alpha_0.55/cleaned_graphs/accepted_seq_only_qwen3-32b_250000-300000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/llm_kg/alpha_0.55/cleaned_gr

In [140]:
united_dataset = united_dataset.to_pandas()
united_dataset_deduplicated = united_dataset.drop_duplicates(subset=keys_duplicates)

whole_dataset = pd.read_csv(os.path.join(dataset_path, 'qwen32b_beta_0.67.csv'))
whole_dataset_deduplicated = whole_dataset.drop_duplicates(subset=keys_duplicates)

fact_score = len(united_dataset_deduplicated) / len(whole_dataset_deduplicated)
print(round(fact_score, 3))

0.757


In [141]:
len(whole_dataset_deduplicated), len(whole_dataset)

(580703, 580703)

In [130]:
dataset_path = '/projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/llm_kg/alpha_0.55/cleaned_graphs'
datasets = []
suffices = [
'accepted_qwen3-32b_0-50000_zero_shot',
'accepted_qwen3-32b_50000-100000_zero_shot',
'accepted_qwen3-32b_100000-150000_zero_shot',
'accepted_qwen3-32b_150000-200000_zero_shot',
'accepted_qwen3-32b_200000-250000_zero_shot',
'accepted_qwen3-32b_250000-300000_zero_shot',
'accepted_qwen3-32b_300000-350000_zero_shot',
'accepted_qwen3-32b_350000-400000_zero_shot',
'accepted_qwen3-32b_400000-450000_zero_shot',
'accepted_qwen3-32b_450000-500000_zero_shot',
'accepted_qwen3-32b_500000-550000_zero_shot',
'accepted_qwen3-32b_550000-580703_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/llm_kg/alpha_0.55/cleaned_graphs/accepted_qwen3-32b_0-50000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/llm_kg/alpha_0.55/cleaned_graphs/accepted_qwen3-32b_50000-100000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/llm_kg/alpha_0.55/cleaned_graphs/accepted_qwen3-32b_100000-150000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/llm_kg/alpha_0.55/cleaned_graphs/accepted_qwen3-32b_150000-200000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/llm_kg/alpha_0.55/cleaned_graphs/accepted_qwen3-32b_200000-250000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/llm_kg/alpha_0.55/cleaned_graphs/accepted_qwen3-32b_250000-300000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/llm_kg/alpha_0.55/cleaned_graphs/accepted_qwen3-32b_300000-350000_zero_shot
loaded

In [131]:
united_dataset = united_dataset.to_pandas()
united_dataset_deduplicated = united_dataset.drop_duplicates(subset=keys_duplicates)

whole_dataset = pd.read_csv(os.path.join(dataset_path, 'qwen32b_beta_0.67.csv'))
whole_dataset_deduplicated = whole_dataset.drop_duplicates(subset=keys_duplicates)

fact_score = len(united_dataset_deduplicated) / len(whole_dataset_deduplicated)
print(round(fact_score, 3))

0.779


In [132]:
len(whole_dataset_deduplicated), len(whole_dataset)

(580703, 580703)

#### Seed KG

In [27]:
dataset_path = '/scratch/gpfs/JHA/mb5157/large_data/diabetes_2025_6years/dataset/for_training/qwen32b/'
datasets = []
suffices = [
'accepted_qwen3-32b_0-28533_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /scratch/gpfs/JHA/mb5157/large_data/diabetes_2025_6years/dataset/for_training/qwen32b/accepted_qwen3-32b_0-28533_zero_shot
13958


In [28]:
13958 / 28533

0.48918795780324537

In [27]:
dataset_path = '/scratch/gpfs/JHA/mb5157/large_data/diabetes_2025_6years/dataset/for_training/qwen32b/'
datasets = []
suffices = [
'accepted_qwen3-32b_0-28533_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /scratch/gpfs/JHA/mb5157/large_data/diabetes_2025_6years/dataset/for_training/qwen32b/accepted_qwen3-32b_0-28533_zero_shot
13958


In [29]:
dataset_path = '/scratch/gpfs/JHA/mb5157/large_data/diabetes_2025_6years/dataset/for_training/qwen32b/'
datasets = []
suffices = [
'accepted_seq_only_qwen3-32b_0-28533_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /scratch/gpfs/JHA/mb5157/large_data/diabetes_2025_6years/dataset/for_training/qwen32b/accepted_seq_only_qwen3-32b_0-28533_zero_shot
13204


In [30]:
13204 / 28533

0.4627624154487786

### Qwen 14B LLM KG

In [42]:
dataset_path = '/scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen14b_fixed_rel_kg/'
datasets = []
suffix = 'accepted_seq_only_qwen3-32b'
suffices = []

dataset_len = 100_000
step = 50000
for i in range(0, dataset_len, step):
    last = dataset_len if i + step > dataset_len else i + step
    suffices.append(f'{suffix}_{i}-{last}')
    
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen14b_fixed_rel_kg/accepted_seq_only_qwen3-32b_0-50000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen14b_fixed_rel_kg/accepted_seq_only_qwen3-32b_50000-100000
46394


In [63]:
dataset_path = '/scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen14b_fixed_rel_kg/'
datasets = []
suffix = 'accepted_qwen3-32b'
suffices = []

dataset_len = 100_000
step = 20000
for i in range(0, dataset_len, step):
    last = dataset_len if i + step > dataset_len else i + step
    suffices.append(f'{suffix}_{i}-{last}')
    
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen14b_fixed_rel_kg/accepted_qwen3-32b_0-20000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen14b_fixed_rel_kg/accepted_qwen3-32b_20000-40000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen14b_fixed_rel_kg/accepted_qwen3-32b_40000-60000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen14b_fixed_rel_kg/accepted_qwen3-32b_60000-80000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen14b_fixed_rel_kg/accepted_qwen3-32b_80000-100000
45744


### Grok

In [24]:
dataset_path = '/scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg'
datasets = []
suffix = 'accepted_seq_only_qwen3-32b'
suffices = []

dataset_len = 100_000
step = 50_000
for i in range(0, dataset_len, step):
    last = dataset_len if i + step > dataset_len else i + step
    suffices.append(f'{suffix}_{i}-{last}_zero_shot')
    
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg/accepted_seq_only_qwen3-32b_0-50000_zero_shot
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg/accepted_seq_only_qwen3-32b_50000-100000_zero_shot
79410


In [25]:
united_dataset

Dataset({
    features: ['first_ids_text_num', 'head', 'relation', 'tail', 'text', '__index_level_0__'],
    num_rows: 79410
})

In [59]:
dataset_path = '/scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg'
datasets = []
suffix = 'accepted_qwen3-32b'
suffices = []

dataset_len = 100_000
step = 10_000
for i in range(0, dataset_len, step):
    last = dataset_len if i + step > dataset_len else i + step
    suffices.append(f'{suffix}_{i}-{last}')
    
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg/accepted_qwen3-32b_0-10000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg/accepted_qwen3-32b_10000-20000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg/accepted_qwen3-32b_20000-30000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg/accepted_qwen3-32b_30000-40000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg/accepted_qwen3-32b_40000-50000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg/accepted_qwen3-32b_50000-60000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg/accepted_qwen3-32b_60000-70000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg/accepted_qwen3-32b_70000-80000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg/accepted_qwen3-32b_80000-90000
loaded from /scratch/gpfs/JHA/mb

## Gemini evaluation

#### GraphMERT

In [68]:
dataset_path = '/projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/'
datasets = []
suffix = 'accepted_seq_only_gemini-2.0-flash'
suffices = []

dataset_len = 139_565
step = 1000
for i in range(0, dataset_len, step):
    last = dataset_len if i + step > dataset_len else i + step
    suffices.append(f'{suffix}_{i}-{last}')
    
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/accepted_seq_only_gemini-2.0-flash_0-1000
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/accepted_seq_only_gemini-2.0-flash_1000-2000
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/accepted_seq_only_gemini-2.0-flash_2000-3000
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/accepted_seq_only_gemini-2.0-flash_3000-4000
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/accepted_seq_only_gemini-2.0-flash_4000-5000
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/accepted_seq_only_gemini-2.0-flash_5000-6000
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/accepted_seq_only_gemini-2.0-flash_6000-7000
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/accepted_seq_only_gemini-2.0-flash_7000-8000
loa

In [69]:
for key_word in ('yes', 'no', 'maybe', ''):
    # "" -- including missing values
    dataset_keyword = united_dataset.filter(lambda ex: ex["response"] == [key_word],
                                 desc=f"Keep only {key_word}")
    
    print(len(dataset_keyword))
    print(f'{key_word}:', round(len(dataset_keyword)/len(united_dataset), 3))
    print()

Keep only yes: 100%|██████████| 139565/139565 [00:01<00:00, 109566.88 examples/s]


111167
yes: 0.797



Keep only no: 100%|██████████| 139565/139565 [00:01<00:00, 110014.69 examples/s]


25562
no: 0.183



Keep only maybe: 100%|██████████| 139565/139565 [00:01<00:00, 114227.06 examples/s]


0
maybe: 0.0



Keep only : 100%|██████████| 139565/139565 [00:01<00:00, 110868.75 examples/s]

2836
: 0.02



In [70]:
dataset_path = '/projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/'
datasets = []
suffix = 'accepted_gemini-2.0-flash'
suffices = []

dataset_len = 139_565
step = 1000
for i in range(0, dataset_len, step):
    last = dataset_len if i + step > dataset_len else i + step
    suffices.append(f'{suffix}_{i}-{last}')
    
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/accepted_gemini-2.0-flash_0-1000
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/accepted_gemini-2.0-flash_1000-2000
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/accepted_gemini-2.0-flash_2000-3000
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/accepted_gemini-2.0-flash_3000-4000
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/accepted_gemini-2.0-flash_4000-5000
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/accepted_gemini-2.0-flash_5000-6000
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/accepted_gemini-2.0-flash_6000-7000
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/accepted_gemini-2.0-flash_7000-8000
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_gr

In [71]:
for key_word in ('yes', 'no', 'maybe', ''):
    # "" -- including missing values
    dataset_keyword = united_dataset.filter(lambda ex: ex["response"] == [key_word],
                                 desc=f"Keep only {key_word}")
    
    print(len(dataset_keyword))
    print(f'{key_word}:', round(len(dataset_keyword)/len(united_dataset), 3))
    print()

Keep only yes: 100%|██████████| 139565/139565 [00:01<00:00, 94983.15 examples/s] 


117227
yes: 0.84



Keep only no: 100%|██████████| 139565/139565 [00:01<00:00, 109477.80 examples/s]


17892
no: 0.128



Keep only maybe: 100%|██████████| 139565/139565 [00:01<00:00, 113613.33 examples/s]


0
maybe: 0.0



Keep only : 100%|██████████| 139565/139565 [00:01<00:00, 110049.17 examples/s]

4446
: 0.032



#### LLM

In [65]:
dataset_path = '/scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen32b_fixed_rel_kg/alpha_0.55'
datasets = []
suffix = 'accepted_seq_only_gemini-2.0-flash'
suffices = []

dataset_len = 515460
step = 1000
for i in range(0, dataset_len, step):
    last = dataset_len if i + step > dataset_len else i + step
    suffices.append(f'{suffix}_{i}-{last}')
    
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen32b_fixed_rel_kg/alpha_0.55/accepted_seq_only_gemini-2.0-flash_0-1000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen32b_fixed_rel_kg/alpha_0.55/accepted_seq_only_gemini-2.0-flash_1000-2000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen32b_fixed_rel_kg/alpha_0.55/accepted_seq_only_gemini-2.0-flash_2000-3000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen32b_fixed_rel_kg/alpha_0.55/accepted_seq_only_gemini-2.0-flash_3000-4000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen32b_fixed_rel_kg/alpha_0.55/accepted_seq_only_gemini-2.0-flash_4000-5000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen32b_fixed_rel_kg/alpha_0.55/accepted_seq_only_gemini-2.0-flash_5000-6000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen32b_fixed_rel_kg/alpha_0.55/accepted_seq_only_gemini-2.0-flash_6000-7000
loaded fr

In [66]:
for key_word in ('yes', 'no', 'maybe', ''):
    # "" -- including missing values
    dataset_keyword = united_dataset.filter(lambda ex: ex["response"] == [key_word],
                                 desc=f"Keep only {key_word}")
    
    print(len(dataset_keyword))
    print(f'{key_word}:', round(len(dataset_keyword)/len(united_dataset), 3))
    print()

Keep only yes: 100%|██████████| 515460/515460 [00:17<00:00, 28807.71 examples/s]


253291
yes: 0.491



Keep only no: 100%|██████████| 515460/515460 [00:07<00:00, 67016.41 examples/s]


214941
no: 0.417



Keep only maybe: 100%|██████████| 515460/515460 [00:07<00:00, 71168.95 examples/s]


0
maybe: 0.0



Keep only : 100%|██████████| 515460/515460 [00:07<00:00, 67923.38 examples/s]

47228
: 0.092



In [25]:
253291 / (515460)

0.4913882745508866

In [74]:
# seq + general knowledge LLM
united_dataset = load_from_disk('/scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen32b_fixed_rel_kg/alpha_0.55/accepted_gemini-2.0-flash_all')

In [75]:
for key_word in ('yes', 'no', 'maybe', ''):
    # "" -- including missing values
    dataset_keyword = united_dataset.filter(lambda ex: ex["response"] == [key_word],
                                 desc=f"Keep only {key_word}")
    
    print(len(dataset_keyword))
    print(f'{key_word}:', round(len(dataset_keyword)/len(united_dataset), 3))
    print()

Keep only yes: 100%|██████████| 515460/515460 [00:14<00:00, 36022.15 examples/s]


228453
yes: 0.443



Keep only no: 100%|██████████| 515460/515460 [00:07<00:00, 67757.98 examples/s]


155925
no: 0.302



Keep only maybe: 100%|██████████| 515460/515460 [00:07<00:00, 71134.94 examples/s]


0
maybe: 0.0



Keep only : 100%|██████████| 515460/515460 [00:07<00:00, 69385.63 examples/s]

131082
: 0.254



### Grok

In [7]:
# seq + general knowledge LLM
united_dataset = load_from_disk('/scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg/accepted_seq_only_gemini-2.0-flash_all')

In [8]:
for key_word in ('yes', 'no',''):
    # "" -- including missing values
    dataset_keyword = united_dataset.filter(lambda ex: ex["response"] == [key_word],
                                 desc=f"Keep only {key_word}")
    
    print(len(dataset_keyword))
    print(f'{key_word}:', round(len(dataset_keyword)/len(united_dataset), 3))
    print()

Keep only yes: 100%|██████████| 100000/100000 [00:02<00:00, 39605.48 examples/s]


73269
yes: 0.733



Keep only no: 100%|██████████| 100000/100000 [00:01<00:00, 61020.78 examples/s]


8005
no: 0.08



Keep only : 100%|██████████| 100000/100000 [00:01<00:00, 68964.59 examples/s]

18726
: 0.187



In [9]:
# seq + general knowledge LLM
united_dataset = load_from_disk('/scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg/accepted_gemini-2.0-flash_all')

In [10]:
for key_word in ('yes', 'no', ''):
    # "" -- including missing values
    dataset_keyword = united_dataset.filter(lambda ex: ex["response"] == [key_word],
                                 desc=f"Keep only {key_word}")
    
    print(len(dataset_keyword))
    print(f'{key_word}:', round(len(dataset_keyword)/len(united_dataset), 3))
    print()

Keep only yes: 100%|██████████| 100000/100000 [00:02<00:00, 40791.47 examples/s]


67919
yes: 0.679



Keep only no: 100%|██████████| 100000/100000 [00:01<00:00, 70303.05 examples/s]


6550
no: 0.066



Keep only : 100%|██████████| 100000/100000 [00:01<00:00, 69814.24 examples/s]

25531
: 0.255



### Qwen14B

In [64]:

united_dataset = load_from_disk('/scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen14b_fixed_rel_kg/accepted_seq_only_gemini-2.0-flash_all')

In [65]:
for key_word in ('yes', 'no', ''):
    # "" -- including missing values
    dataset_keyword = united_dataset.filter(lambda ex: ex["response"] == [key_word],
                                 desc=f"Keep only {key_word}")
    
    print(len(dataset_keyword))
    print(f'{key_word}:', round(len(dataset_keyword)/len(united_dataset), 3))
    print()

Keep only yes: 100%|██████████| 100000/100000 [00:02<00:00, 39178.78 examples/s]


62397
yes: 0.624



Keep only no: 100%|██████████| 100000/100000 [00:01<00:00, 67797.83 examples/s]


20439
no: 0.204



Keep only : 100%|██████████| 100000/100000 [00:01<00:00, 64341.95 examples/s]

17164
: 0.172



In [66]:
# seq + general knowledge LLM
united_dataset = load_from_disk('/scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen14b_fixed_rel_kg/accepted_gemini-2.0-flash_all')

In [67]:
for key_word in ('yes', 'no', ''):
    # "" -- including missing values
    dataset_keyword = united_dataset.filter(lambda ex: ex["response"] == [key_word],
                                 desc=f"Keep only {key_word}")
    
    print(len(dataset_keyword))
    print(f'{key_word}:', round(len(dataset_keyword)/len(united_dataset), 3))
    print()

Keep only yes: 100%|██████████| 100000/100000 [00:02<00:00, 38833.30 examples/s]


57584
yes: 0.576



Keep only no: 100%|██████████| 100000/100000 [00:01<00:00, 66840.16 examples/s]


18250
no: 0.182



Keep only : 100%|██████████| 100000/100000 [00:01<00:00, 68011.13 examples/s]

24166
: 0.242



# Validity score

In [11]:
# keys to drop duplicates
keys_duplicates = ['id', 'head', 'relation', 'tail']

In [12]:
from collections import Counter

def filter_yes_only(united_dataset: Dataset) -> Dataset:
    return united_dataset.filter(
        lambda x: [v == "yes" for v in x["accepted"]],
        batched=True
    )

### main results

In [16]:
dataset_path = '/projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/'
datasets = []
suffices = [
'validated_qwen3-32b_0-50000_zero_shot',
'validated_qwen3-32b_50000-100000_zero_shot',
'validated_qwen3-32b_100000-139565_zero_shot',

]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))


loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/validated_qwen3-32b_0-50000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/validated_qwen3-32b_50000-100000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/validated_qwen3-32b_100000-139565_zero_shot
139565


In [17]:
united_dataset = united_dataset.to_pandas()
united_dataset_no_duplicates = united_dataset.drop_duplicates(subset=keys_duplicates)
c = Counter(united_dataset_no_duplicates['accepted'])
length = len(united_dataset_no_duplicates['accepted'])
counts = {k: round(c.get(k, 0) / length, 3) for k in ('yes', 'maybe', 'no')}
print(len(united_dataset_no_duplicates), len(united_dataset))
print(counts)

138201 139565
{'yes': 0.687, 'maybe': 0.183, 'no': 0.108}


In [25]:
to_save = united_dataset_no_duplicates[united_dataset_no_duplicates['accepted'] == 'yes'].drop(columns=['accepted', 'Unnamed: 0'])
to_save.to_csv('/projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/validated_yes_no_duplicates.csv', index=False)

#### Qwen3 32B kg

In [5]:
dataset_path = '/projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55'
datasets = []
suffices = [
'validated_qwen3-32b_0-100000_zero_shot',
'validated_qwen3-32b_100000-200000_zero_shot',
'validated_qwen3-32b_200000-300000_zero_shot',
'validated_qwen3-32b_300000-400000_zero_shot',
'validated_qwen3-32b_400000-500000_zero_shot',
'validated_qwen3-32b_500000-515460_zero_shot',

]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

c = Counter(united_dataset['accepted'])
length = len(united_dataset['accepted'])
counts = {k: round(c.get(k, 0) / length, 3) for k in ('yes', 'maybe', 'no', '')}
print(counts)

loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/validated_qwen3-32b_0-100000_zero_shot
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/validated_qwen3-32b_100000-200000_zero_shot
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/validated_qwen3-32b_200000-300000_zero_shot
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/validated_qwen3-32b_300000-400000_zero_shot
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/validated_qwen3-32b_400000-500000_zero_shot
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/validated_qwen3-32b_500000-515460_zero_shot
515460
{'yes': 0.43, 'maybe': 0.241, 'no': 0.314, '': 0.015}


In [17]:
filtered_dataset = filter_yes_only(united_dataset)
print(len(filtered_dataset))
filtered_dataset.remove_columns('accepted').save_to_disk('/projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/validated_yes')

Filter: 100%|██████████| 515460/515460 [00:00<00:00, 806380.92 examples/s]


221632


Saving the dataset (5/5 shards): 100%|██████████| 221632/221632 [00:09<00:00, 23211.19 examples/s]


### ablations

In [180]:
dataset_path = '/projects/JHA/shared/dataset/qwen32b/alpha_0.55/mlm/cleaned_graphs'
datasets = []
suffices = [
'validated_qwen3-32b_0-100000_zero_shot',
'validated_qwen3-32b_100000-188211_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

# c = Counter(united_dataset['verdict'])
# length = len(united_dataset['verdict'])
# counts = {k: round(c.get(k, 0) / length, 3) for k in ('yes', 'maybe', 'no')}
# print(counts)

loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/mlm/cleaned_graphs/validated_qwen3-32b_0-100000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/mlm/cleaned_graphs/validated_qwen3-32b_100000-188211_zero_shot
188211


In [181]:
united_dataset = united_dataset.to_pandas()
check = pd.read_csv('/projects/JHA/shared/dataset/qwen32b/alpha_0.55/mlm/cleaned_graphs/qwen32b_beta_0.67.csv')

cols = ["head", "relation", "tail"]
assert len(check) == len(united_dataset_pd)
mask = check[cols].eq(united_dataset_pd[cols].values).all(axis=1)
all_match = bool(mask.all())
num_mismatch = int((~mask).sum())
assert num_mismatch == 0

out = check.merge(
    united_dataset_pd, left_index=True, right_index=True, suffixes=("", "_right")
)

out_no_duplicates = out.drop_duplicates(subset=keys_duplicates)
out_no_duplicates = out_no_duplicates.drop(['head_right', 'relation_right', 'tail_right'], axis=1)

c = Counter(out_no_duplicates['verdict'])
length = len(out_no_duplicates['verdict'])
print(length)
counts = {k: round(c.get(k, 0) / length, 3) for k in ('yes', 'maybe', 'no')}
print(counts)

185768
{'yes': 0.693, 'maybe': 0.182, 'no': 0.103}


In [184]:
dataset_path = '/projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_dropout/cleaned_graphs'
datasets = []
suffices = [
    'validated_qwen3-32b_0-50000_zero_shot',
    'validated_qwen3-32b_50000-100000_zero_shot',
    'validated_qwen3-32b_100000-138363_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

c = Counter(united_dataset['verdict'])
length = len(united_dataset['verdict'])
counts = {k: round(c.get(k, 0) / length, 3) for k in ('yes', 'maybe', 'no')}
print(counts)

loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_dropout/cleaned_graphs/validated_qwen3-32b_0-50000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_dropout/cleaned_graphs/validated_qwen3-32b_50000-100000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_dropout/cleaned_graphs/validated_qwen3-32b_100000-138363_zero_shot
138363
{'yes': 0.66, 'maybe': 0.192, 'no': 0.12}


In [185]:
dataset_path = '/projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_hgat/cleaned_graphs'
datasets = []
suffices = [
    'validated_qwen3-32b_0-50000_zero_shot',
    'validated_qwen3-32b_50000-100000_zero_shot',
    'validated_qwen3-32b_100000-150000_zero_shot',
    'validated_qwen3-32b_150000-155311_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

c = Counter(united_dataset['verdict'])
length = len(united_dataset['verdict'])
counts = {k: round(c.get(k, 0) / length, 3) for k in ('yes', 'maybe', 'no')}
print(counts)

loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_hgat/cleaned_graphs/validated_qwen3-32b_0-50000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_hgat/cleaned_graphs/validated_qwen3-32b_50000-100000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_hgat/cleaned_graphs/validated_qwen3-32b_100000-150000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_hgat/cleaned_graphs/validated_qwen3-32b_150000-155311_zero_shot
155311
{'yes': 0.675, 'maybe': 0.175, 'no': 0.123}


In [26]:
dataset_path = '/scratch/gpfs/JHA/mb5157/large_data/diabetes_2025_6years/dataset/for_training/qwen32b'
datasets = []
suffices = [
    'validated_seed_0.55_qwen3-32b_0-28533_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

c = Counter(united_dataset['verdict'])
length = len(united_dataset['verdict'])
counts = {k: round(c.get(k, 0) / length, 3) for k in ('yes', 'maybe', 'no')}
print(counts)

loaded from /scratch/gpfs/JHA/mb5157/large_data/diabetes_2025_6years/dataset/for_training/qwen32b/validated_seed_0.55_qwen3-32b_0-28533_zero_shot
28533
{'yes': 0.534, 'maybe': 0.1, 'no': 0.347}


### Iteration 2 ablations

In [189]:
dataset_path = '/projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/expanded_kg/alpha_0.55/cleaned_graphs'
datasets = []
suffices = [
'validated_qwen3-32b_0-200000_zero_shot',
'validated_qwen3-32b_200000-400000_zero_shot',
'validated_qwen3-32b_400000-600000_zero_shot',
'validated_qwen3-32b_600000-614503_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

c = Counter(united_dataset['verdict'])
length = len(united_dataset['verdict'])
counts = {k: round(c.get(k, 0) / length, 3) for k in ('yes', 'maybe', 'no')}
print(counts)

loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/expanded_kg/alpha_0.55/cleaned_graphs/validated_qwen3-32b_0-200000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/expanded_kg/alpha_0.55/cleaned_graphs/validated_qwen3-32b_200000-400000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/expanded_kg/alpha_0.55/cleaned_graphs/validated_qwen3-32b_400000-600000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/expanded_kg/alpha_0.55/cleaned_graphs/validated_qwen3-32b_600000-614503_zero_shot
614503
{'yes': 0.586, 'maybe': 0.309, 'no': 0.08}


In [188]:
dataset_path = '/projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/llm_kg/alpha_0.55/cleaned_graphs'
datasets = []
suffices = [
'validated_qwen3-32b_0-200000_zero_shot',
'validated_qwen3-32b_200000-400000_zero_shot',
'validated_qwen3-32b_400000-580703_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

c = Counter(united_dataset['verdict'])
length = len(united_dataset['verdict'])
counts = {k: round(c.get(k, 0) / length, 3) for k in ('yes', 'maybe', 'no')}
print(counts)

loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/llm_kg/alpha_0.55/cleaned_graphs/validated_qwen3-32b_0-200000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/llm_kg/alpha_0.55/cleaned_graphs/validated_qwen3-32b_200000-400000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/llm_kg/alpha_0.55/cleaned_graphs/validated_qwen3-32b_400000-580703_zero_shot
580703
{'yes': 0.586, 'maybe': 0.305, 'no': 0.084}


In [190]:
dataset_path = '/projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/expanded_kg/alpha_0.55/cleaned_graphs'
datasets = []
suffices = [
'validated_qwen3-32b_0-200000_zero_shot',
'validated_qwen3-32b_200000-400000_zero_shot',
'validated_qwen3-32b_400000-492271_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

c = Counter(united_dataset['verdict'])
length = len(united_dataset['verdict'])
counts = {k: round(c.get(k, 0) / length, 3) for k in ('yes', 'maybe', 'no')}
print(counts)

loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/expanded_kg/alpha_0.55/cleaned_graphs/validated_qwen3-32b_0-200000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/expanded_kg/alpha_0.55/cleaned_graphs/validated_qwen3-32b_200000-400000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/expanded_kg/alpha_0.55/cleaned_graphs/validated_qwen3-32b_400000-492271_zero_shot
492271
{'yes': 0.651, 'maybe': 0.24, 'no': 0.081}


In [191]:
dataset_path = '/projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/llm_kg/alpha_0.55/cleaned_graphs'
datasets = []
suffices = [
'validated_qwen3-32b_0-200000_zero_shot',
'validated_qwen3-32b_200000-400000_zero_shot',
'validated_qwen3-32b_400000-589338_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

c = Counter(united_dataset['verdict'])
length = len(united_dataset['verdict'])
counts = {k: round(c.get(k, 0) / length, 3) for k in ('yes', 'maybe', 'no')}
print(counts)

loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/llm_kg/alpha_0.55/cleaned_graphs/validated_qwen3-32b_0-200000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/llm_kg/alpha_0.55/cleaned_graphs/validated_qwen3-32b_200000-400000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/seed_kg_ablations/before_cleaning/llm_kg/alpha_0.55/cleaned_graphs/validated_qwen3-32b_400000-589338_zero_shot
589338
{'yes': 0.58, 'maybe': 0.309, 'no': 0.087}


### Fact score after validity score

In [9]:
dataset_path = '/projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/'
datasets = []
suffices = [
'validated_qwen3-32b_0-50000_zero_shot',
'validated_qwen3-32b_50000-100000_zero_shot',
'validated_qwen3-32b_100000-139565_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/validated_qwen3-32b_0-50000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/validated_qwen3-32b_50000-100000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/validated_qwen3-32b_100000-139565_zero_shot
139565


In [13]:
united_dataset_yes = filter_yes_only(united_dataset)

In [14]:
united_dataset_yes = united_dataset_yes.to_pandas()
united_dataset_yes_deduplicated = united_dataset_yes.drop_duplicates(subset=keys_duplicates)
print(len(united_dataset_yes_deduplicated))

95085


In [ ]:
### del it

In [8]:
dataset_path = '/projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/validated_yes.csv'
tmp = pd.read_csv(dataset_path)
tmp

,Unnamed: 0,id,text,head,relation,tail
0,0,413486,"to the quartiles of urinary albumin excretion ( q1 [ < 7. 7 mg / g ], 17. 1 μg / ml ; q2 [ 7. 7 – 15. 0 mg / g ], 17. 5 μg / ml ; q3 [ 15. 1 – 61. 4 mg / g ], 18. 6 μg / ml ; q4 [ ≥61. 5 mg / g ], 22. 3 μg / ml ; p for trend = 0. 003 ) under adjustment with other covariates. a positive correlation was found between plasma lrg1 level and urinary albumin excretion ( ρ = 0. 256, p < 0.",plasma lrg1 level,associated_with,urinary albumin excretion
1,1,450824,"the aim of this article is to assess the association of subclinical atherosclerosis with accelerated epigenetic age and to investigate the underlying mechanisms mediating this association. methods and results whole blood methylomics, transcriptomics, and plasma proteomics were obtained for 391 participants of the progression of early subclinical atherosclerosis study. epigenetic age was calculated from methylomics data for each participant. its divergence from chronological age is termed epigenetic age acceleration. subclinical atherosclerosis burden was estimated by multi - territory 2d / 3d vascular ultrasound and by coronary artery calcification. in healthy individuals, the presence, extension, and progression of subclinical atherosclerosis were associated with a significant acceleration of the grim epigenetic age, a",epigenetic age acceleration,associated_with,subclinical atherosclerosis
2,2,406192,"several hallmark processes of cancer. over the past decades, researchers have identified the epithelial protein lost in neoplasm ( eplin, also known as lima1 ) as a key regulator of cytoskeletal dynamics, cytoskeletal organization, motility, as well as cell growth and metabolism. dysregulation of eplin is implicated in various aspects of cancer progression, such as tumor growth, invasion, metastasis, and therapeutic resistance. its altered expression levels or activity can disrupt cytoskeletal dynamics, leading to aberrant cell motility and invasiveness characteristic of malignant cells. moreover, the involvement of eplin in cell growth and metabolism underscores its significance in orchestrating key processes essential for cancer cell survival and","epithelial protein lost in neoplasm ( eplin, also known as lima1 )",plays_role,cytoskeleton
3,4,771977,"by a decay in cellular functions and proliferation, resulting in increased cellular damage and death. this condition plays an essential role in the aging process and significantly contributes to the development of age - related complications. on the other hand, ferroptosis is a systemic cell death pathway characterized by excessive iron accumulation followed by the generation of reactive oxygen species ( ros ). oxidative stress is a common trigger of this condition and may be induced by various factors such as toxins, drugs, and inflammation. ferroptosis is linked to numerous disorders, including cardiovascular disease, neurodegeneration, and cancer. senescence is believed to contribute to the decay in tissue and organ functions occurring with",senescence,associated_with,ferroptosis
4,7,33486,"in diabetes distress ( all p > 0. 05 ). when change in diabetes acceptance from baseline to follow - up was added to the model, improved diabetes distress was explained by increasing diabetes acceptance ( β = 0. 41, p < 0. 01 ) and a shorter duration of diabetes ( β = - 0. 18, p = 0. 03 ), while baseline diabetes acceptance was no longer significantly associated ( β = −0. 14, p > 0. 05 ). conclusions diabetes acceptance is inversely related to diabetes distress, and increasing acceptance explained greater improvement in diabetes distress. these findings suggest that increasing diabetes acceptance may facilitate the reduction",diabetes acceptance,associated_with,diabetes distress
...,...,...,...,...,...,...
96015,139558,163609,"were included in the study population ( n = 2074 ). protein levels were analyzed using the olink proseek multiplex inflammation panel. percentage body fat ( % bf ) an

In [ ]:
## del it

In [220]:
dataset_path = '/projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/validated_yes'
datasets = []
suffices = [
'accepted_qwen3-32b_0-50000_zero_shot',
'accepted_qwen3-32b_50000-96020_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
whole_dataset = concatenate_datasets(datasets)
print('with duplicates:', len(whole_dataset))
whole_dataset = whole_dataset.to_pandas()

whole_dataset_deduplicated = whole_dataset.drop_duplicates(subset=keys_duplicates)
print('without duplicates:', len(whole_dataset_deduplicated))

fact_score = len(whole_dataset_deduplicated) / len(united_dataset_yes_deduplicated)
print(round(fact_score, 3))

loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/validated_yes/accepted_qwen3-32b_0-50000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/validated_yes/accepted_qwen3-32b_50000-96020_zero_shot
with duplicates: 76989
without duplicates: 76271
0.802


In [222]:
dataset_path = '/projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/validated_yes'
datasets = []
suffices = [
'accepted_seq_only_qwen3-32b_0-50000_zero_shot',
'accepted_seq_only_qwen3-32b_50000-96020_zero_shot',
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
whole_dataset = concatenate_datasets(datasets)
print('with duplicates:', len(whole_dataset))
whole_dataset = whole_dataset.to_pandas()

whole_dataset_deduplicated = whole_dataset.drop_duplicates(subset=keys_duplicates)
print('without duplicates:', len(whole_dataset_deduplicated))

fact_score = len(whole_dataset_deduplicated) / len(united_dataset_yes_deduplicated)
print(round(fact_score, 3))

loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/validated_yes/accepted_seq_only_qwen3-32b_0-50000_zero_shot
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/validated_yes/accepted_seq_only_qwen3-32b_50000-96020_zero_shot
with duplicates: 73828
without duplicates: 73166
0.769


In [73]:
dataset_path = '/projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/validated_yes'
datasets = []
suffices = [
'accepted_qwen3-32b_0-75000_zero_shot',
'accepted_qwen3-32b_75000-150000_zero_shot',
'accepted_qwen3-32b_150000-221632_zero_shot'
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/validated_yes/accepted_qwen3-32b_0-75000_zero_shot
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/validated_yes/accepted_qwen3-32b_75000-150000_zero_shot
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/validated_yes/accepted_qwen3-32b_150000-221632_zero_shot
159312


In [50]:
159312 / 221632

0.7188131677736067

In [35]:
dataset_path = '/projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/validated_yes'
datasets = []
suffices = [
'accepted_seq_only_qwen3-32b_0-75000_zero_shot',
'accepted_seq_only_qwen3-32b_75000-150000_zero_shot',
'accepted_seq_only_qwen3-32b_150000-221632_zero_shot'
]
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/validated_yes/accepted_seq_only_qwen3-32b_0-75000_zero_shot
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/validated_yes/accepted_seq_only_qwen3-32b_75000-150000_zero_shot
loaded from /projects/JHA/shared/graph/qwen32b_fixed_rel_kg/alpha_0.55/validated_yes/accepted_seq_only_qwen3-32b_150000-221632_zero_shot
123322


In [36]:
123322 / 221632

0.5564268697660988

In [43]:
united_dataset = load_from_disk('/scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen14b_fixed_rel_kg/validated_qwen3-32b_0-100000')

In [44]:
for key_word in ('yes', 'no', 'maybe', ''):
    # "" -- including missing values
    dataset_keyword = united_dataset.filter(lambda ex: ex["verdict"] == key_word,
                                 desc=f"Keep only {key_word}")
    
    print(len(dataset_keyword))
    print(f'{key_word}:', round(len(dataset_keyword)/len(united_dataset), 3))
    print()

Keep only yes: 100%|██████████| 100000/100000 [00:00<00:00, 211525.97 examples/s]


31847
yes: 0.318



Keep only no: 100%|██████████| 100000/100000 [00:00<00:00, 216348.44 examples/s]


44421
no: 0.444



Keep only maybe: 100%|██████████| 100000/100000 [00:00<00:00, 218557.72 examples/s]


21916
maybe: 0.219



Keep only : 100%|██████████| 100000/100000 [00:00<00:00, 220908.06 examples/s]

1816
: 0.018



In [46]:
0.444 + 0.018

0.462

### Grok

In [23]:
united_dataset = load_from_disk('/scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg/validated_qwen3-32b_0-100000_zero_shot')

for key_word in ('yes', 'no', 'maybe', ''):
    # "" -- including missing values
    dataset_keyword = united_dataset.filter(lambda ex: ex["verdict"] == key_word,
                                 desc=f"Keep only {key_word}")
    
    print(len(dataset_keyword))
    print(f'{key_word}:', round(len(dataset_keyword)/len(united_dataset), 3))
    print()

56914
yes: 0.569

18351
no: 0.184

21948
maybe: 0.219

2787
: 0.028



In [28]:
# no:
1 - 0.569 - 0.219

0.21200000000000005

In [26]:
united_dataset = load_from_disk('/scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg/validated_qwen3-32b_0-100000_zero_shot')


dataset_keyword = united_dataset.filter(lambda ex: ex["verdict"] == 'no',
                             desc=f"Keep only {key_word}")

dataset_keyword_pd = dataset_keyword.to_pandas()
dataset_keyword_pd

Keep only : 100%|██████████| 100000/100000 [00:00<00:00, 217531.20 examples/s]


,head,relation,tail,verdict
0,obesity,possibly_equivalent_to,type 2 diabetes mellitus,no
1,diabetes p4p program,associated_with,hyperglycemic hyperosmolar state,no
2,avpd,associated_with,cardiovascular death,no
3,rpl9,associated_with,dna methylation,no
4,type 2 diabetes,part_of,protocol-based diabetes care,no
...,...,...,...,...
18346,finerenone,cause_of,chronic kidney disease,no
18347,high carbohydrate diet,occurs_in,type 1 diabetes,no
18348,epiallopregnanolone sulfate,has_method,trpm3,no
18349,dialbetes step,associated_with,lipid parameters,no


In [31]:
dataset_keyword_pd.iloc[:50]

,head,relation,tail,verdict
0,obesity,possibly_equivalent_to,type 2 diabetes mellitus,no
1,diabetes p4p program,associated_with,hyperglycemic hyperosmolar state,no
2,avpd,associated_with,cardiovascular death,no
3,rpl9,associated_with,dna methylation,no
4,type 2 diabetes,part_of,protocol-based diabetes care,no
5,cnut,associated_with,sbp,no
6,bariatric surgery,cause_of,obesity,no
7,phloretamide,associated_with,glucose-6-phosphatase,no
8,metabolic syndrome,has_component,diabetes,no
9,vancomycin,causative_agent_of,glucose disorder,no


In [29]:
test = load_from_disk('/scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg/triples_with_chunks_sampled').to_pandas()

In [30]:
test.iloc[:50]

,first_ids_text_num,head,relation,tail,text,__index_level_0__
0,0,type 2 diabetes mellitus,associated_with,antihypertensive medication,"micelle formulation with an average micelle size of 123.6 ± 3.1 nm and a monodisperse size distribution, showing a polydispersity index value of 0.215 ± 0.021. The proper nanoparticles were co-formulated with sodium alginate as a biologically decomposing and safe-to-administer biopolymer. The Box–Behnken factorial design ensured proper design space development, where the optimal sodium alginate bead formulation had a uniform, extended-release drug release mechanism similar to commercially available tablet preparations. The main conclusion is that the rapid-burst-like drug release can be h...",131836
1,24,diabetes,cause_of,cardiovascular disease,"oids. The ability to provide a better understanding of the molecular targets and mechanism of action of C. nutans extracts and their phytochemical components should encourage future studies to develop new therapeutic strategies for better use of this herb to combat inflammatory diseases.\n\nThe aim of the present study was to compare the epidemiological patterns of severe acute respiratory syndrome coronavirus 2 (SARS-CoV2) infections, hospitalizations, deaths, and duration of hospitalization during the fourth, fifth and sixth epidemic waves of coronavirus disease 2019 (COVID-19) in Iran. ...",7668
2,0,temporal bisection task,method_of,time perception,"2 related factors were associated with a severe disease course.\n\nPatients with autoimmune polyendocrinopathy syndrome type 1 (APS-1) caused by autosomal recessive AIRE deficiency display autoantibodies (auto-Abs) neutralizing type I IFNs, conferring a predisposition to life-threatening COVID-19 pneumonia. We report that patients with autosomal recessive NIK or RelB deficiency, or a specific type of autosomal dominant (AD) NF-κB2 deficiency also display neutralizing auto-Abs against type I IFNs. They are prone to severe viral disease, including life-threatening COVID-19 pneumonia, influen...",38708
3,69,obesity,possibly_equivalent_to,type 2 diabetes mellitus,"< 0.001). The frequency of multivitamin users among COVID-19 participants was significantly lower than non-users (44.4 vs. 55.6; p < 0.003). Male respondents within 26–35 years of age were more likely to use multivitamin supplements than females (30.1 vs. 22.6%; p < 0.05) of same age group. Predictors for DS use were increased age group, income, education level and COVID-19 status. In conclusion, an increase in unhealthy diet behavior was observed among Saudi males and females during the pandemic lockdown and the predictors of DS use included increased age, income, education level and COV...",3628
4,0,cardiovascular diseases,associated_with,type 2 diabetes mellitus,"7 words) and were written at a mean grade level of 13.7 (95% CI, 13.4-14.1). Among 71 privacy policies, 69 (97.2%; 95% CI, 91.4%-99.5%) addressed types of user information automatically collected by the website, 70 (98.6%; 95% CI, 93.8%-99.9%) addressed how collected information would be used, 66 (93.0%; 95% CI, 85.3%-97.5%) addressed categories of third-party recipients of user information, and 40 (56.3%; 95% CI, 44.5%-67.7%) named specific third-party companies or services receiving user information. Conclusions and Relevance In this cross-sectional study of hospital website privacy poli...",48966
5,0,venous thromboembolism (vte),associated_with,diabetes,", which is likely to cause further damage to ocular surface. Anti-inflammatory therapy may help protect ocular surface in postmenopausal women with PANDO.\n\nSeveral lines of evidence have suggested that patients following venous thromboembolism (VTE) are at higher risk of arterial thromboembolism (ATE). Prothrombotic fibrin clot characteristics were reported in individuals with cardiovascular risk factors. We investigated whether specific fibrin clot properties measured after 3–4 months of anticoagulation characterize VTE patients w

### Qwen4 with NLP filtering

In [74]:
united_dataset = load_from_disk('/projects/JHA/shared/dataset/combined_with_qwen4b/validated_qwen3-32b_0-9000')
united_dataset

Dataset({
    features: ['head', 'relation', 'tail', 'verdict'],
    num_rows: 9000
})

In [76]:
for key_word in ('yes', 'no', 'maybe', ''):
    # "" -- including missing values
    dataset_keyword = united_dataset.filter(lambda ex: ex["verdict"] == key_word,
                                 desc=f"Keep only {key_word}")
    
    print(len(dataset_keyword))
    print(f'{key_word}:', round(len(dataset_keyword)/len(united_dataset), 3))
    print()

Keep only yes: 100%|██████████| 9000/9000 [00:00<00:00, 182335.50 examples/s]


5740
yes: 0.638



Keep only no: 100%|██████████| 9000/9000 [00:00<00:00, 186632.86 examples/s]


1356
no: 0.151



Keep only maybe: 100%|██████████| 9000/9000 [00:00<00:00, 183388.73 examples/s]


1661
maybe: 0.185



Keep only : 100%|██████████| 9000/9000 [00:00<00:00, 194404.75 examples/s]

243
: 0.027



## Gemini evaluation

#### GraphMERT

In [56]:
dataset_path = '/projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/'
datasets = []
suffix = 'validated_gemini-2.0-flash'
suffices = []

dataset_len = 139_565
for i in range(0, dataset_len, 1000):
    last = dataset_len if i + 1000 > dataset_len else i + 1000
    suffices.append(f'{suffix}_{i}-{last}')
    
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/validated_gemini-2.0-flash_0-1000
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/validated_gemini-2.0-flash_1000-2000
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/validated_gemini-2.0-flash_2000-3000
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/validated_gemini-2.0-flash_3000-4000
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/validated_gemini-2.0-flash_4000-5000
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/validated_gemini-2.0-flash_5000-6000
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/validated_gemini-2.0-flash_6000-7000
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/validated_gemini-2.0-flash_7000-8000
loaded from /projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cl

In [5]:
united_dataset = load_from_disk('/projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/validated_gemini-2.0-flash_all')

for key_word in ('yes', 'no', 'maybe', ''):
    # "" -- including missing values
    dataset_keyword = united_dataset.filter(lambda ex: ex["response"] == [key_word],
                                 desc=f"Keep only {key_word}")
    
    print(len(dataset_keyword))
    print(f'{key_word}:', round(len(dataset_keyword)/len(united_dataset), 3))
    print()

Keep only yes: 100%|██████████| 139565/139565 [00:01<00:00, 106138.04 examples/s]


113426
yes: 0.813



Keep only no: 100%|██████████| 139565/139565 [00:01<00:00, 110378.90 examples/s]


327
no: 0.002



Keep only maybe: 100%|██████████| 139565/139565 [00:01<00:00, 92917.50 examples/s]


25766
maybe: 0.185



Keep only : 100%|██████████| 139565/139565 [00:01<00:00, 112463.87 examples/s]

46
: 0.0



In [33]:
103220 + 34868 + 541 + 936

139565

#### seed KG

In [8]:
united_dataset = load_from_disk('/projects/JHA/shared/dataset/qwen32b/alpha_0.55/validated_gemini-2.0-flash_all')

for key_word in ('yes', 'no', 'maybe', ''):
    # "" -- including missing values
    dataset_keyword = united_dataset.filter(lambda ex: ex["response"] == [key_word],
                                 desc=f"Keep only {key_word}")
    
    print(len(dataset_keyword))
    print(f'{key_word}:', round(len(dataset_keyword)/len(united_dataset), 3))
    print()

Keep only yes: 100%|██████████| 28533/28533 [00:00<00:00, 132930.73 examples/s]


23526
yes: 0.825



Keep only no: 100%|██████████| 28533/28533 [00:00<00:00, 139945.67 examples/s]


311
no: 0.011



Keep only maybe: 100%|██████████| 28533/28533 [00:00<00:00, 135052.93 examples/s]


4695
maybe: 0.165



Keep only : 100%|██████████| 28533/28533 [00:00<00:00, 146064.71 examples/s]

1
: 0.0



### Qwen32B LLM KG

In [63]:
dataset_path = '/scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen32b_fixed_rel_kg/alpha_0.55/'
datasets = []
suffix = 'validated_gemini-2.0-flash'
suffices = []

dataset_len = 515460
step = 1000
for i in range(0, dataset_len, step):
    last = dataset_len if i + step > dataset_len else i + step
    suffices.append(f'{suffix}_{i}-{last}')
    
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen32b_fixed_rel_kg/alpha_0.55/validated_gemini-2.0-flash_0-1000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen32b_fixed_rel_kg/alpha_0.55/validated_gemini-2.0-flash_1000-2000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen32b_fixed_rel_kg/alpha_0.55/validated_gemini-2.0-flash_2000-3000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen32b_fixed_rel_kg/alpha_0.55/validated_gemini-2.0-flash_3000-4000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen32b_fixed_rel_kg/alpha_0.55/validated_gemini-2.0-flash_4000-5000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen32b_fixed_rel_kg/alpha_0.55/validated_gemini-2.0-flash_5000-6000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen32b_fixed_rel_kg/alpha_0.55/validated_gemini-2.0-flash_6000-7000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/

In [9]:
united_dataset = load_from_disk('/scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen32b_fixed_rel_kg/alpha_0.55/validated_gemini-2.0-flash_all')

In [10]:
for key_word in ('yes', 'no', 'maybe', ''):
    # "" -- including missing values
    dataset_keyword = united_dataset.filter(lambda ex: ex["response"] == [key_word],
                                 desc=f"Keep only {key_word}")
    
    print(len(dataset_keyword))
    print(f'{key_word}:', round(len(dataset_keyword)/len(united_dataset), 3))
    print()

Keep only yes: 100%|██████████| 515460/515460 [00:13<00:00, 38210.78 examples/s]


340372
yes: 0.66



Keep only no: 100%|██████████| 515460/515460 [00:07<00:00, 66525.73 examples/s]


1654
no: 0.003



Keep only maybe: 100%|██████████| 515460/515460 [00:07<00:00, 68686.77 examples/s]


173319
maybe: 0.336



Keep only : 100%|██████████| 515460/515460 [00:07<00:00, 68693.12 examples/s]

115
: 0.0



### Grok LLM KG

In [16]:
dataset_path = '/scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg'
datasets = []
suffix = 'validated_gemini-2.0-flash'
suffices = []

dataset_len = 100_000
step = 1000
for i in range(0, dataset_len, step):
    last = dataset_len if i + step > dataset_len elbse i + step
    suffices.append(f'{suffix}_{i}-{last}')
    
for suf in suffices:
    path = os.path.join(dataset_path, suf)
    dataset = load_from_disk(path)            
    print(f'loaded from {path}')
    datasets.append(dataset)
united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))

loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg/validated_gemini-2.0-flash_0-1000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg/validated_gemini-2.0-flash_1000-2000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg/validated_gemini-2.0-flash_2000-3000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg/validated_gemini-2.0-flash_3000-4000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg/validated_gemini-2.0-flash_4000-5000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg/validated_gemini-2.0-flash_5000-6000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg/validated_gemini-2.0-flash_6000-7000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg/validated_gemini-2.0-flash_7000-8000
loaded from /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/grok_llm_kg/validated_gemi

In [17]:
for key_word in ('yes', 'no', 'maybe', ''):
    # "" -- including missing values
    dataset_keyword = united_dataset.filter(lambda ex: ex["response"] == [key_word],
                                 desc=f"Keep only {key_word}")
    
    print(len(dataset_keyword))
    print(f'{key_word}:', round(len(dataset_keyword)/len(united_dataset), 3))
    print()

68827
yes: 0.688

303
no: 0.003

30855
maybe: 0.309

15
: 0.0



### Qwen14B

In [68]:
united_dataset = load_from_disk('/scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen14b_fixed_rel_kg/validated_gemini-2.0-flash_all')

In [70]:
for key_word in ('yes', 'no', 'maybe', ''):
    # "" -- including missing values
    dataset_keyword = united_dataset.filter(lambda ex: ex["response"] == [key_word],
                                 desc=f"Keep only {key_word}")
    
    print(len(dataset_keyword))
    print(f'{key_word}:', round(len(dataset_keyword)/len(united_dataset), 3))
    print()

Keep only yes: 100%|██████████| 100000/100000 [00:03<00:00, 33145.32 examples/s]


53356
yes: 0.534



Keep only no: 100%|██████████| 100000/100000 [00:01<00:00, 68582.86 examples/s]


930
no: 0.009



Keep only maybe: 100%|██████████| 100000/100000 [00:01<00:00, 68522.61 examples/s]


45673
maybe: 0.457



Keep only : 100%|██████████| 100000/100000 [00:01<00:00, 70473.29 examples/s]

41
: 0.0

